In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
from IPython.display import Audio
from scipy.io import wavfile
import scipy.io
import soundfile as sf
import tensorflow as tf
import tensorflow_hub as hub
import warnings
warnings.filterwarnings('ignore')
from tensorflow.keras import layers, models
import librosa
import re



2025-08-25 09:07:44.585812: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2025-08-25 09:07:44.585841: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2025-08-25 09:07:44.585857: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2025-08-25 09:07:44.590352: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
# read the whole directory of data and produce all waveforms + labels
def parseAudioData(data_path):
    audio_data = []
    for i in os.listdir(data_path):
        filename = data_path+i
        filename = filename.format(i=i)
        for j in os.listdir(filename):
            path = os.path.join(filename, j)
            if os.path.isfile(path) and path.lower().endswith('.wav'):
                try:
                    waveform, sr = librosa.load(path, sr=16000)
                    audio_data.append([waveform, i])
                except Exception as e:
                    print(f"Error loading {path}: {e}")
    return audio_data

audio_data = parseAudioData("960ms_data/training_data/")

In [3]:
audio_dataframe = pd.DataFrame(audio_data, columns=["audio_data", "class"])
for i in audio_dataframe["audio_data"].to_list():
    print(len(i))


15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
15360
1536

In [4]:
def pad_waveform(waveform, target_length):
    if len(waveform) < target_length:
        return np.pad(waveform, (0, target_length - len(waveform)))
    return waveform


In [5]:
#waveform data
def preprocess(audio_dataframe):
    audio_dataframe["audio_data"] = audio_dataframe["audio_data"].apply(lambda x: pad_waveform(x, 15360))
    audio_dataframe["audio_data"] = audio_dataframe["audio_data"].apply(lambda x: x[:15360])
    audio_data = audio_dataframe["audio_data"].to_list()
    audio_data = np.array(audio_data)
    #audio_data = 10 * np.log10(np.abs(audio_data) + 1e-10) * audio_data / np.abs(audio_data)
    return audio_data

audio_data = preprocess(audio_dataframe)

In [6]:
audio_data

array([[-0.02380371, -0.1078186 ,  0.05673218, ..., -0.03863525,
        -0.19491577,  0.11920166],
       [ 0.00057983, -0.03979492,  0.04544067, ...,  0.03353882,
         0.05218506, -0.07327271],
       [-0.06997681,  0.04708862, -0.02069092, ..., -0.15298462,
         0.01211548,  0.15414429],
       ...,
       [-0.0090332 , -0.00750732, -0.00527954, ..., -0.08065796,
        -0.07073975, -0.03985596],
       [-0.00036621, -0.00045776, -0.00210571, ..., -0.00784302,
        -0.00802612, -0.00961304],
       [ 0.00030518,  0.00140381,  0.00289917, ...,  0.00418091,
         0.00320435,  0.00265503]], dtype=float32)

In [7]:
def getMelSpectrograms(audio_data):
    # Constants
    sample_rate = 16000  # Hz
    duration = 0.96      # seconds
    n_fft = 400          # ~25ms window
    hop_length = 160     # ~10ms stride (gives 96 frames over 0.96s)
    n_mels = 64
    target_length = 15600

    spectrograms = []
    for waveform in audio_data:
        # Load or generate waveform
        # waveform: 1D array with length 15360
        # Example: load from file or generate dummy
        #waveform, _ = librosa.load("audio.wav", sr=sample_rate, duration=duration)
        waveform = pad_waveform(waveform, target_length)
        
        # Compute mel spectrogram
        mel_spec = librosa.feature.melspectrogram(
            y=waveform,
            sr=sample_rate,
            n_fft=n_fft,
            hop_length=hop_length,
            n_mels=n_mels,
            center=False,
            power=2.0  # power=2.0 for power spectrogram (as YAMNet expects)
        )
        
        # Convert to log scale (optional, if model expects log-mel)
        log_mel_spec = librosa.power_to_db(mel_spec, ref=np.max)
        spectrograms.append(log_mel_spec)

    return spectrograms

audio_data = getMelSpectrograms(audio_data)

In [9]:
#one hot encoder for labels
def OneHotEncoder(df):
    class_names = ["background", "water", "singing", "abnormal"]
    
    # Create a lookup table
    table = tf.lookup.StaticVocabularyTable(
        tf.lookup.KeyValueTensorInitializer(class_names, tf.range(len(class_names), dtype=tf.int64)),
        num_oov_buckets=1
    )
    
    # Convert the "class" column to a Tensor
    label_tensor = tf.constant(df["class"].to_list())
    
    label_indices = table.lookup(label_tensor)
    return tf.one_hot(label_indices, depth=len(class_names))

In [10]:
classes = OneHotEncoder(audio_dataframe)

2025-08-25 09:07:48.795977: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2025-08-25 09:07:48.795992: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:168] retrieving CUDA diagnostic information for host: 4050770f3584
2025-08-25 09:07:48.795996: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:175] hostname: 4050770f3584
2025-08-25 09:07:48.796077: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:199] libcuda reported version is: 555.42.2
2025-08-25 09:07:48.796088: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:203] kernel reported version is: NOT_FOUND: could not find kernel module information in driver version file contents: "NVRM version: NVIDIA UNIX Open Kernel Module for x86_64  555.42.02  Release Build  (dvs-builder@U16-I3-A03-02-4)  Mon May 13 17:10:33 UTC 2024
GCC version:  gcc version 11.4.0 (Ubuntu 11.4

In [11]:
#format data in tf dataset, shuffle and batch
dataset = tf.data.Dataset.from_tensor_slices((audio_data, classes))
dataset

<_TensorSliceDataset element_spec=(TensorSpec(shape=(64, 96), dtype=tf.float32, name=None), TensorSpec(shape=(4,), dtype=tf.float32, name=None))>

In [12]:
dataset = dataset.shuffle(1000)
dataset = dataset.repeat(2)
dataset_batched = dataset.batch(16)

In [13]:
#create validation set
val_size = int(0.2 * len(dataset_batched))

val_ds = dataset_batched.take(val_size)
train_ds = dataset_batched.skip(val_size)

In [14]:
#yamnet256 = tf.keras.models.load_model('yamnet_256_64x96_tl.h5/')
yamnet1024 = tf.keras.models.load_model('yamnet_1024_64x96_tl.h5/')

In [15]:
#yamnet256.summary()
yamnet1024.summary()

Model: "model_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 96, 1)]       0         
                                                                 
 permute (Permute)           (None, 96, 64, 1)         0         
                                                                 
 model (Functional)          (None, 1024)              3217344   
                                                                 
 new_head (Dense)            (None, 10)                10250     
                                                                 
Total params: 3227594 (12.31 MB)
Trainable params: 10250 (40.04 KB)
Non-trainable params: 3217344 (12.27 MB)
_________________________________________________________________


In [16]:
# Keep all layers exept the classification head
#new_input = yamnet256.input
new_input = yamnet1024.input

#x = yamnet256.get_layer("permute")(new_input)
#x = yamnet256.get_layer("sequential_2")(x)
x = yamnet1024.get_layer("permute")(new_input)
x = yamnet1024.get_layer("model")(x)


In [17]:
#CUSTOM MODEL

# Add your own classification layers
#x = tf.keras.layers.Dense(64, activation='relu')(yamnet_embeddings)
#x = tf.keras.layers.Dropout(0.3)(x)
#outputs = tf.keras.layers.Dense(3, activation='softmax')(x)

x = layers.Flatten()(x)
x = layers.Dense(16, activation='relu')(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(16, activation='relu')(x)
x = layers.Dropout(0.1)(x)
x = layers.Dense(16, activation='relu')(x)
outputs = layers.Dense(4, activation='softmax')(x)

model = tf.keras.Model(inputs=new_input, outputs=outputs)

model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 64, 96, 1)]       0         
                                                                 
 permute (Permute)           (None, 96, 64, 1)         0         
                                                                 
 model (Functional)          (None, 1024)              3217344   
                                                                 
 flatten (Flatten)           (None, 1024)              0         
                                                                 
 dense (Dense)               (None, 16)                16400     
                                                                 
 dropout (Dropout)           (None, 16)                0         
                                                                 
 dense_1 (Dense)             (None, 16)                272   

In [18]:
#early stopping to prevent overfitting
early_stopping = tf.keras.callbacks.EarlyStopping(
    monitor='val_loss',      # What to monitor
    patience=5,              # How many epochs to wait after no improvement
    restore_best_weights=True  # Restore best model weights
)

In [19]:
model.compile(optimizer='adam', loss='categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_ds, validation_data=val_ds, epochs=20, callbacks=[early_stopping])

Epoch 1/20
96/96 [==============================] - 4s 18ms/step - loss: 1.5983 - accuracy: 0.3373 - val_loss: 1.1497 - val_accuracy: 0.5924
Epoch 2/20
96/96 [==============================] - 1s 11ms/step - loss: 1.0478 - accuracy: 0.6301 - val_loss: 0.8843 - val_accuracy: 0.7690
Epoch 3/20
96/96 [==============================] - 1s 11ms/step - loss: 0.8500 - accuracy: 0.6980 - val_loss: 0.5991 - val_accuracy: 0.7582
Epoch 4/20
96/96 [==============================] - 1s 11ms/step - loss: 0.5972 - accuracy: 0.7686 - val_loss: 0.4544 - val_accuracy: 0.8207
Epoch 5/20
96/96 [==============================] - 1s 11ms/step - loss: 0.5075 - accuracy: 0.7980 - val_loss: 0.4805 - val_accuracy: 0.8125
Epoch 6/20
96/96 [==============================] - 1s 11ms/step - loss: 0.5053 - accuracy: 0.7817 - val_loss: 0.3973 - val_accuracy: 0.8397
Epoch 7/20
96/96 [==============================] - 1s 11ms/step - loss: 0.5030 - accuracy: 0.7784 - val_loss: 0.4560 - val_accuracy: 0.7826
Epoch 8/20
96

In [20]:
print(model.input_shape, model.output_shape)

(None, 64, 96, 1) (None, 4)


In [21]:
test_data = parseAudioData("960ms_data/testing_data/")
test_dataframe = pd.DataFrame(test_data, columns=["audio_data", "class"])
test_data = preprocess(test_dataframe)
test_data = getMelSpectrograms(test_data)
test_data = tf.constant(test_data)
test_classes = OneHotEncoder(test_dataframe)
test_dataset = tf.data.Dataset.from_tensor_slices((test_data, test_classes))
test_dataset = test_dataset.batch(1)

In [22]:
test_dataset

<_BatchDataset element_spec=(TensorSpec(shape=(None, 64, 96), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [23]:
model.evaluate(test_dataset)

307/307 [==============================] - 1s 4ms/step - loss: 1.0743 - accuracy: 0.5472


[1.0742897987365723, 0.5472312569618225]

In [24]:
predictions = model.predict(test_dataset)

for i in range(len(predictions)):
    print(f"Sample {i}: True = {test_classes[i]}, Predicted = {predictions[i]}")

307/307 [==============================] - 1s 3ms/step
Sample 0: True = [0. 1. 0. 0.], Predicted = [2.3049342e-03 9.9766076e-01 3.3646604e-05 6.0305160e-07]
Sample 1: True = [0. 1. 0. 0.], Predicted = [4.4933358e-01 5.4046959e-01 1.0024017e-02 1.7279510e-04]
Sample 2: True = [0. 1. 0. 0.], Predicted = [1.6463954e-02 9.8298061e-01 5.2366540e-04 3.1787953e-05]
Sample 3: True = [0. 1. 0. 0.], Predicted = [8.9960426e-01 1.7513666e-02 8.2289651e-02 5.9240567e-04]
Sample 4: True = [0. 1. 0. 0.], Predicted = [9.1218008e-03 9.9062496e-01 2.4301380e-04 1.0219434e-05]
Sample 5: True = [0. 1. 0. 0.], Predicted = [9.6296072e-03 9.9023658e-01 1.3051204e-04 3.3340227e-06]
Sample 6: True = [0. 1. 0. 0.], Predicted = [3.6947286e-01 6.2111074e-01 9.1535933e-03 2.6276303e-04]
Sample 7: True = [0. 1. 0. 0.], Predicted = [1.0438986e-02 9.8927981e-01 2.6882588e-04 1.2437319e-05]
Sample 8: True = [0. 1. 0. 0.], Predicted = [3.5272557e-02 9.6373361e-01 9.4705744e-04 4.6890567e-05]
Sample 9: True = [0. 1. 0. 

TENSORFLOW to TFLITE

In [25]:

base_name = "yamnet1024"
save_dir = "saved_models"
os.makedirs(save_dir, exist_ok=True)


# === FIND NEXT MODEL NUMBER ===
def get_next_model_path(base_dir, base_name):
    pattern = re.compile(rf"{re.escape(base_name)}_(\d+)")
    existing = [
        d for d in os.listdir(base_dir)
        if os.path.isdir(os.path.join(base_dir, d)) and pattern.match(d)
    ]

    numbers = [
        int(pattern.match(d).group(1)) for d in existing
        if pattern.match(d)
    ]
    next_num = max(numbers) + 1 if numbers else 1
    model_name = f"{base_name}_{next_num:03d}"
    return os.path.join(base_dir, model_name)

model_path = get_next_model_path(save_dir, base_name)

tf.saved_model.save(model, model_path)

INFO:tensorflow:Assets written to: saved_models/yamnet1024_010/assets


INFO:tensorflow:Assets written to: saved_models/yamnet1024_010/assets


In [26]:
converter = tf.lite.TFLiteConverter.from_saved_model("saved_models/yamnet1024_010")


In [27]:
#def representative_dataset():
#    # Take 100 input samples (ignore labels)
#    calibration_data = [x.numpy() for x, _ in dataset.take(100)]
#    for spectrogram in calibration_data:
#        # Ensure shape [1, 64, 96, 1]
#        spectrogram = np.expand_dims(spectrogram, axis=0).astype(np.float32)
#        spectrogram = np.expand_dims(spectrogram, axis=-1).astype(np.float32)
#        yield [spectrogram]

In [28]:
# quantize to int 8
#converter.optimizations = [tf.lite.Optimize.DEFAULT]
#converter.representative_dataset = representative_dataset
#converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS_INT8]
#converter.inference_input_type = tf.int8  # or tf.uint8
#converter.inference_output_type = tf.int8  # or tf.uint8
#tflite_quant_model = converter.convert()

In [29]:
# no quantizations
converter.optimizations = [tf.lite.Optimize.DEFAULT]
tflite_quant_model = converter.convert()

2025-08-25 09:08:30.669762: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:378] Ignored output_format.
2025-08-25 09:08:30.669784: W tensorflow/compiler/mlir/lite/python/tf_tfl_flatbuffer_helpers.cc:381] Ignored drop_control_dependency.
2025-08-25 09:08:30.670021: I tensorflow/cc/saved_model/reader.cc:83] Reading SavedModel from: saved_models/yamnet1024_010
2025-08-25 09:08:30.677487: I tensorflow/cc/saved_model/reader.cc:51] Reading meta graph with tags { serve }
2025-08-25 09:08:30.677510: I tensorflow/cc/saved_model/reader.cc:146] Reading SavedModel debug info (if present) from: saved_models/yamnet1024_010
2025-08-25 09:08:30.693709: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:382] MLIR V1 optimization pass is not enabled
2025-08-25 09:08:30.699874: I tensorflow/cc/saved_model/loader.cc:233] Restoring SavedModel bundle.
2025-08-25 09:08:30.847509: I tensorflow/cc/saved_model/loader.cc:217] Running initialization op on SavedModel bundle at path: sa

In [30]:
# Save the model to a .tflite file
with open("yamnet1024_010.tflite", "wb") as f:
    f.write(tflite_quant_model)
